# Testing spaCy Framework

In [ ]:
!pip install -U spacy

     |████████████████████████████████| 12.8MB 284kB/s 
     |████████████████████████████████| 1.1MB 49.0MB/s 
     |████████████████████████████████| 460kB 35.4MB/s 
     |████████████████████████████████| 9.1MB 36.3MB/s 
     |████████████████████████████████| 122kB 57.8MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=12b8affed7957cf934a6299a6c6cb45a0bac0ad9fe2bd46510c336422dced2ba
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: smart-open 4.2.0
    Uninstallin

In [ ]:
!python -m spacy download en_core_web_sm

2021-04-11 09:37:09.182668: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.7MB 326kB/s 
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

Noun phrases: ['Sebastian Thrun', 'self-driving cars', 'Google', 'few people', 'the company', 'him', 'I', 'you', 'very senior CEOs', 'major American car companies', 'my hand', 'I', 'Thrun', 'an interview', 'Recode']
Verbs: ['start', 'work', 'drive', 'take', 'can', 'tell', 'would', 'shake', 'turn', 'talk', 'say']
Sebastian NORP
Google ORG
2007 DATE
American NORP
Recode ORG
earlier this week DATE


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def get_entity(text):
  # output all the entity that text contains
  doc = nlp(text)
  for entity in doc.ents:
    print(entity.text, entity.label_)

In [ ]:
texts = ["Apache Kafka 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.2.0, 2.2.1, or 2.3.0 are configured with one or more config providers",
        "Users should upgrade to 2.2.2 or higher, or 2.3.1 or higher where this vulnerability has been fixed.",
         "In Apache Kafka versions between 0.11.0.0 and 2.1.0, it is possible to manually craft a Produce request which bypasses transaction/idempotent ACL validation.",
         "Potential HTTP request smuggling in Apache Tomcat",
         "OpenStack Identity (Keystone) Folsom 2012.2.4 and earlier, Grizzly before 2013.1.1, and Havana does not immediately revoke the authentication token when deleting a user through the Keystone v2 API, which allows remote authenticated users to retain access via the token."]

print("==========")
for text in texts:
  get_entity(text)
  print("==========")

Apache Kafka PERSON
2.0.1 DATE
2.1.1 DATE
2.2.0 DATE
2.2.1 DATE
2.3.0 DATE
one CARDINAL
2.2.2 CARDINAL
2.3.1 CARDINAL
Apache Kafka PRODUCT
2.1.0 CARDINAL
ACL ORG
OpenStack Identity (Keystone ORG
2012.2.4 CARDINAL
Havana GPE
the Keystone v2 API FAC


# Importing Input Data

In [ ]:
IMPORTERS = ['debian']
DATA_DIR = '/content/drive/MyDrive/GsoC 2021/Data/data/'

In [ ]:
sentences = []
for importer in IMPORTERS:
  f = open(DATA_DIR + importer + '.txt', 'r')
  sentences.extend(f.read().split('@')) # needs to be small
print(len(sentences))
print(sentences)

# Sample Code of Transfer Learning on spaCy Model

In [ ]:
'''
TP = Word predicted as either Package_Name or Package_Version and present in the data(train/test/validation) as either Package_Name or Package_Version 
FP = Word predicted as either Package_Name or Package_Version and not present in the data (train/test/validation) as either Package_Name or Package_Version 
FN = Word present in the data(train/test/validation) data as either Package_Name or Package_Version but not predicted as either Package_Name or Package_Version
'''
# Precision = TP/(TP+FP )
def calc_precision(pred, true):        
    precision = len([x for x in pred if x in true]) / (len(pred) + 1e-20)
    return precision

# Recall = TP/(TP+FN)
def calc_recall(pred, true):
    recall = len([x for x in true if x in pred]) / (len(true) + 1e-20)
    return recall

# F1-score = 2 * Precision* Recall/ (Precision+Recall)
def calc_f1(precision, recall):
    f1 = 2 * ((precision * recall) / (precision + recall + 1e-20))
    return f1

In [ ]:
# run the predictions on each sentence in the evaluation  dataset, and return the metrics
def evaluate(ner, data):
    preds = [ner(x[0]) for x in data]

    precisions, recalls, f1s = [], [], []

    # iterate over predictions and test data and calculate precision, recall, and F1-score
    for pred, true in zip(preds, data):
        true = [x[2] for x in list(chain.from_iterable(true[1].values()))]
        pred = [i.label_ for i in pred.ents]
        precision = calc_precision(true, pred)
        precisions.append(precision)
        recall = calc_recall(true, pred)
        recalls.append(recall)
        f1s.append(calc_f1(precision, recall))

    return {"textcat_p": np.mean(precisions), "textcat_r": np.mean(recalls), "textcat_f":np.mean(f1s)}

In [ ]:
def train_spacy(train_data, labels, iterations, dropout = 0.5, display_freq = 1):
    ''' Train a spacy NER model, which can be queried against with test data
   
    train_data : training data in the format of (sentence, {entities: [(start, end, label)]})
    labels : a list of unique annotations
    iterations : number of training iterations
    dropout : dropout proportion for training
    display_freq : number of epochs between logging losses to console
    '''

    valid_f1scores=[]
    test_f1scores=[]
    nlp = spacy.load("en_core_web_md")

    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
    else:
        ner = nlp.get_pipe("ner")
   
    # Add entity labels to the NER pipeline
    for i in labels:
        ner.add_label(i)

    # Disable other pipelines in SpaCy to only train NER
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itr in range(iterations):
            random.shuffle(train_data) # shuffle the training data before each iteration
            losses = {}
            batches = minibatch(train_data, size = compounding(16.0, 64.0, 1.5))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(          
                    texts,
                    annotations,
                    drop = dropout,  
                    sgd = optimizer,
                    losses = losses)
            
            print('=======================================')
            print('Interation = '+str(itr))
            print('Losses = '+str(losses))

            # Evaluate present model on Validation dataset
            scores = evaluate(nlp,VALID_DATA)
            valid_f1scores.append(scores["textcat_f"])
            print('===============VALID DATA========================')
            print('F1-score = '+str(scores["textcat_f"]))
            print('Precision = '+str(scores["textcat_p"]))
            print('Recall = '+str(scores["textcat_r"]))

            # evaluate present model on Test dataset
            scores = evaluate(nlp,TEST_DATA)
            test_f1scores.append(scores["textcat_f"])
            print('===============TEST DATA========================')
            print('F1-score = '+str(scores["textcat_f"]))
            print('Precision = '+str(scores["textcat_p"]))
            print('Recall = '+str(scores["textcat_r"]))
            print('=======================================')
            
    return nlp,valid_f1scores,test_f1scores

# Training